In [1]:
import os
import zipfile
import urllib

import numpy as np
import pandas as pd
import geopandas as gpd
import pylab as pl

from __future__ import print_function
import sys
import matplotlib.pylab as pl
%pylab inline

from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from scipy.spatial.distance import cdist, pdist
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.mixture import GMM

Populating the interactive namespace from numpy and matplotlib


In [6]:
!for ((y=93; y<=99; y+=1)); do wget \
https://www2.census.gov/Econ2001_And_Earlier/CBP_CSV//zbp$y\totals.zip; done

!for ((y=0; y<=1; y+=1)); do wget \
https://www2.census.gov/Econ2001_And_Earlier/CBP_CSV//zbp0$y\totals.zip; done

!for ((y=2; y<=9; y+=1)); do wget \
https://www2.census.gov/econ200$y\/CBP_CSV/zbp0$y\totals.zip; done

!for ((y=10; y<=14; y+=1)); do wget \
https://www2.census.gov/econ20$y\/CBP_CSV/zbp$y\totals.zip; done

--2017-12-11 16:31:49--  https://www2.census.gov/Econ2001_And_Earlier/CBP_CSV//zbp93totals.zip
Resolving www2.census.gov... 104.106.248.146, 2600:141b:13:288::208c, 2600:141b:13:295::208c
Connecting to www2.census.gov|104.106.248.146|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2017-12-11 16:31:50 ERROR 404: Not Found.

--2017-12-11 16:31:50--  https://www2.census.gov/Econ2001_And_Earlier/CBP_CSV//zbp94totals.zip
Resolving www2.census.gov... 104.106.248.146, 2600:141b:13:288::208c, 2600:141b:13:295::208c
Connecting to www2.census.gov|104.106.248.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 611524 (597K) [application/zip]
Saving to: “zbp94totals.zip”

100%[======================================>] 611,524     --.-K/s   in 0.04s   

2017-12-11 16:31:50 (14.5 MB/s) - “zbp94totals.zip” saved [611524/611524]

--2017-12-11 16:31:50--  https://www2.census.gov/Econ2001_And_Earlier/CBP_CSV//zbp95totals.zip
Resolving www2.census.gov... 

HTTP request sent, awaiting response... 200 OK
Length: 660109 (645K) [application/zip]
Saving to: “zbp09totals.zip”

100%[======================================>] 660,109     --.-K/s   in 0.04s   

2017-12-11 16:31:52 (14.4 MB/s) - “zbp09totals.zip” saved [660109/660109]

--2017-12-11 16:31:52--  https://www2.census.gov/econ2010/CBP_CSV/zbp10totals.zip
Resolving www2.census.gov... 104.106.248.146, 2600:141b:13:295::208c, 2600:141b:13:288::208c
Connecting to www2.census.gov|104.106.248.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 797534 (779K) [application/zip]
Saving to: “zbp10totals.zip”

100%[======================================>] 797,534     --.-K/s   in 0.03s   

2017-12-11 16:31:52 (24.8 MB/s) - “zbp10totals.zip” saved [797534/797534]

--2017-12-11 16:31:52--  https://www2.census.gov/econ2011/CBP_CSV/zbp11totals.zip
Resolving www2.census.gov... 104.106.248.146, 2600:141b:13:295::208c, 2600:141b:13:288::208c
Connecting to www2.census.gov|104.106.2

In [11]:
PUIdata = os.getenv('PUIDATA')

In [12]:
year = []
[year.append(str(i)) for i in range(94,100)]
[year.append('0' + str(i)) for i in range(10)]
[year.append(str(i)) for i in range(10,15)]
year

['94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '00',
 '01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14']

In [13]:
for i in year:
    zf = zipfile.ZipFile('zbp'+ i +'totals.zip')
    business_data = pd.read_csv(zf.open('zbp'+i+'totals.zip'.replace('.zip','.txt')))
    print("Year "+str(i)+'\n')
    print(business_data.head())
    print('\n')

Year 94

    zip         name empflag   emp    qp1      ap  est
0  1001   Agawam, MA     NaN  6198  33601  155158  439
1  1002  Amherst, MA     NaN  6073  28924  127367  450
2  1003  Amherst, MA     NaN    68    367    1604   10
3  1004  Amherst, MA     NaN   210    844    4302   47
4  1005    Barre, MA     NaN   678   3010   13521   92


Year 95

    zip             name empflag   emp    qp1      ap  est
0   501   Holtsville, NY       B     0      0       0    2
1  1000  Springfield, MA     NaN    49    529    2127    4
2  1001       Agawam, MA     NaN  5920  32993  152512  439
3  1002      Amherst, MA     NaN  6153  31645  132699  467
4  1003      Amherst, MA     NaN    89    380    1698   10


Year 96

    zip             name empflag   emp    qp1      ap  est
0   501   Holtsville, NY       B     0      0       0    4
1  1000  Springfield, MA     NaN   161   1494    6671   11
2  1001       Agawam, MA     NaN  6130  37746  171045  433
3  1002      Amherst, MA     NaN  6441  32964  14

Year 14

    zip            name empflag emp_nf    emp qp1_nf    qp1 ap_nf      ap  \
0   501  HOLTSVILLE, NY       B      D      0      D      0     D       0   
1  1001      AGAWAM, MA     NaN      G   9110      G  88574     G  364418   
2  1002     AMHERST, MA     NaN      H  17513      G  99432     G  421772   
3  1003     AMHERST, MA     NaN      G    227      H   1911     H    8026   
4  1004     AMHERST, MA       C      S      0      S      0     H    4921   

   est        city stabbr   cty_name  
0    4  HOLTSVILLE     NY    SUFFOLK  
1  458      AGAWAM     MA    HAMPDEN  
2  560     AMHERST     MA  HAMPSHIRE  
3   18     AMHERST     MA  HAMPSHIRE  
4   13     AMHERST     MA  HAMPSHIRE  




In [16]:
obs = 0
dfs = []
for i in year:
    fname = PUIdata + '/zbp' + i + 'totals.zip'
    df = pd.read_csv(fname)
    df.columns = [c.lower() for c in df.columns]
    print(df.columns)
    obs += len(df)
    dfs.append(df)
df = pd.concat(dfs)
print('Concatenate without data loss: %s'%(len(df) == obs))

Index([u'zip', u'name', u'empflag', u'emp', u'qp1', u'ap', u'est'], dtype='object')
Index([u'zip', u'name', u'empflag', u'emp', u'qp1', u'ap', u'est'], dtype='object')
Index([u'zip', u'name', u'empflag', u'emp', u'qp1', u'ap', u'est'], dtype='object')
Index([u'zip', u'name', u'empflag', u'emp', u'qp1', u'ap', u'est'], dtype='object')
Index([u'zip', u'name', u'empflag', u'emp', u'qp1', u'ap', u'est'], dtype='object')
Index([u'zip', u'name', u'empflag', u'emp', u'qp1', u'ap', u'est'], dtype='object')
Index([u'zip', u'name', u'empflag', u'emp', u'qp1', u'ap', u'est'], dtype='object')
Index([u'zip', u'name', u'empflag', u'emp', u'qp1', u'ap', u'est'], dtype='object')
Index([u'zip', u'name', u'empflag', u'emp', u'qp1', u'ap', u'est'], dtype='object')
Index([u'zip', u'name', u'empflag', u'emp', u'qp1', u'ap', u'est'], dtype='object')
Index([u'zip', u'name', u'empflag', u'emp', u'qp1', u'ap', u'est'], dtype='object')
Index([u'zip', u'name', u'empflag', u'emp', u'qp1', u'ap', u'est'], dtype='o

In [17]:
df.head()

,ap,ap_nf,city,cty_name,emp,emp_nf,empflag,est,name,qp1,qp1_nf,stabbr,zip
0,155158,NaN,NaN,NaN,6198,NaN,NaN,439,"Agawam, MA",33601,NaN,NaN,1001
1,127367,NaN,NaN,NaN,6073,NaN,NaN,450,"Amherst, MA",28924,NaN,NaN,1002
2,1604,NaN,NaN,NaN,68,NaN,NaN,10,"Amherst, MA",367,NaN,NaN,1003
3,4302,NaN,NaN,NaN,210,NaN,NaN,47,"Amherst, MA",844,NaN,NaN,1004
4,13521,NaN,NaN,NaN,678,NaN,NaN,92,"Barre, MA",3010,NaN,NaN,1005


In [19]:
url = "https://data.cityofnewyork.us/download/i8iw-xf4u/application%2Fzip"
urllib.urlretrieve(url, 'NYCZIP.zip')
os.system('unzip NYCZIP.zip -d ' + PUIdata + '/NYCZIP')

256

In [2]:
# locate .shp file
for fname in os.listdir(os.getenv('PUIDATA') + '/NYCZIP'):
    if fname.endswith('shp'):
        shpfile = fname

# read in shapefile
zipshp = gpd.GeoDataFrame.from_file(os.getenv('PUIDATA') + '/NYCZIP/' + shpfile)
print(zipshp.crs)
zipshp.head()

{u'lon_0': -74, u'datum': u'NAD83', u'y_0': 0, u'no_defs': True, u'proj': u'lcc', u'x_0': 300000, u'units': u'us-ft', u'lat_2': 41.03333333333333, u'lat_1': 40.66666666666666, u'lat_0': 40.16666666666666}


,ZIPCODE,BLDGZIP,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,SHAPE_AREA,SHAPE_LEN,geometry
0,11436,0,Jamaica,18681.0,2.269930e+07,NY,Queens,36,081,http://www.usps.com/,0.0,0.0,"POLYGON ((1038098.251871482 188138.3800067157,..."
1,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1001613.712964058 186926.4395172149,..."
2,11212,0,Brooklyn,83866.0,4.197210e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1011174.275535807 183696.33770971, 1..."
3,11225,0,Brooklyn,56527.0,2.369863e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((995908.3654508889 183617.6128015518,..."
4,11218,0,Brooklyn,72280.0,3.686880e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((991997.1134308875 176307.4958601296,..."


In [4]:
zipshp = zipshp[['ZIPCODE', 'geometry']]
zipshp['ZIPCODE'] = pd.to_numeric(zipshp['ZIPCODE'])
zipshp.head()

,ZIPCODE,geometry
0,11436,"POLYGON ((1038098.251871482 188138.3800067157,..."
1,11213,"POLYGON ((1001613.712964058 186926.4395172149,..."
2,11212,"POLYGON ((1011174.275535807 183696.33770971, 1..."
3,11225,"POLYGON ((995908.3654508889 183617.6128015518,..."
4,11218,"POLYGON ((991997.1134308875 176307.4958601296,..."


In [8]:
df = df[['est', '']]

,0
0,94
1,95
2,96
3,97
4,98
